In [39]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from pathlib import Path

colormap = {
    'control': '#002D72',
    'amnesiac': '#008767',
    'ssd': '#A45C98',
    'bad_teacher': '#F1C400',
    'dp': '#A6192E',
}

In [40]:
score_type = 'mia'
task = 'class'

df = pd.read_csv(f'/home/mathepa1/vscode-projects/mirror-mirror/output/cifar10_{task}_unlearning_2024_10_21/{score_type}_results.csv')
score_label = f'{score_type.upper() if score_type not in ("consistency", "randomness") else score_type.capitalize()} Score'
df.head()

,forget_set,run_number,model,score
0,0,0,control,0.3772
1,0,0,ssd,0.2082
2,0,0,bad_teacher,0.0002
3,0,0,amnesiac,0.0162
4,0,1,control,0.3938


In [41]:
data_df = df.copy()
data_df['forget_set'] = data_df['forget_set'].apply(lambda x: x.split('_')[-1] if isinstance(x, str) else str(x))
data_df = data_df[data_df['model'] != 'fisher_forgetting']
data_df.head()

,forget_set,run_number,model,score
0,0,0,control,0.3772
1,0,0,ssd,0.2082
2,0,0,bad_teacher,0.0002
3,0,0,amnesiac,0.0162
4,0,1,control,0.3938


In [42]:
plot_df = data_df[data_df['forget_set'] == '3']
plot_df['run_number'] = plot_df['run_number'].apply(lambda x: str(x + 1))
fig = px.bar(
    plot_df,
    x='run_number',
    y='score',
    color='model',
    labels={
        'run_number': 'Round Number',
        'score': score_label,
        'model': 'Model',
    },
    color_discrete_map=colormap,
    barmode='group',
)
fig.update_xaxes(categoryorder='array', categoryarray=sorted(plot_df['run_number'].unique(), key=(lambda x: int(x))))
fig.show()

/tmp/ipykernel_1044465/1607055311.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
mean_df, std_df = df.groupby(['model', 'forget_set']).mean().drop('run_number', axis=1).reset_index(), df.groupby(['model', 'forget_set']).std().drop('run_number', axis=1).reset_index()
mean_df, std_df = mean_df.rename({'score': 'mean_score'}, axis=1), std_df.rename({'score': 'std_score'}, axis=1)
plot_df = pd.concat([mean_df.set_index(['model', 'forget_set']), std_df.set_index(['model', 'forget_set'])], axis=1).reset_index()
plot_df = plot_df[plot_df['forget_set'] != 'random_7000']
plot_df['forget_set'] = plot_df['forget_set'].apply(lambda x: x.split('_')[-1] if isinstance(x, str) else str(x))
plot_df = plot_df[plot_df['model'] != 'fisher_forgetting']
plot_df
plot_df.head()

,model,forget_set,mean_score,std_score
0,amnesiac,0,0.01520,0.002078
1,amnesiac,1,0.00868,0.002016
2,amnesiac,2,0.02428,0.001850
3,amnesiac,3,0.03528,0.003996
4,amnesiac,4,0.01502,0.001318


In [44]:
fig = px.bar(
    plot_df,
    x='forget_set',
    y='mean_score',
    color='model',
    labels={
        'forget_set': 'Forget Set',
        'mean_score': score_label,
        'model': 'Model',
    },
    color_discrete_map=colormap,
    barmode='group',
    # error_y='std_score',
)
fig.update_xaxes(categoryorder='array', categoryarray=sorted(plot_df['forget_set'].unique(), key=(lambda x: int(x))))
fig.show()